In [24]:
import pandas as pd

In [25]:
df = pd.read_csv('youtube_sentiment.csv')

In [26]:
df

,Index,Comment,Predicted Class
0,1.0,"Just subscribed, downloading this and will wat...",Postive
1,2.0,Could you please do ERPnext tutorial ?,Interrogative
2,3.0,I hate that this is where programming is heade...,Negative
3,4.0,Can you please share this using sharegpt?,Interrogative
4,5.0,The fact this video is 2-3 hours when the non ...,Negative
...,...,...,...
1046,996.0,"Me: Starts learning HTML, CSS, and JavaScript ...",Miscellanous
1047,997.0,"basically, a Blazor alternative that was creat...",Miscellanous
1048,998.0,Cant wait to see browsers getting slower :D,Negative
1049,999.0,Great intro! This is a big step forward imho,Positive


In [ ]:
adsfasfsaf